In [1]:
!pip install flask
from flask import Flask, request, render_template
import numpy as np
import threading
import webbrowser
import pickle
from datetime import datetime
import re

In [2]:
from sklearn.preprocessing import MinMaxScaler

with open('rf_model.pkl', 'rb') as f:
    model = pickle.load(f)

#model_joblib = load('rf_model.joblib')
#print("Modelo carregado com joblib.")

#Load scaler for numeric features  
#with open('scaler.pkl', 'rb') as f:
#    scaler = pickle.load(f)
scaler = MinMaxScaler()

# Ajuste do escalonador aos dados

# Load the LabelEncoder  for 'carrier_type'
with open('carrier_type_encoder.pkl', 'rb') as f:
    carrier_type_encoder = pickle.load(f)

# Load the LabelEncoder  for 'county_of_injury'
with open('county_injury_encoder.pkl', 'rb') as f:
    county_injury_encoder = pickle.load(f)

# Load the LabelEncoder  for 'district_name'
with open('district_name_encoder.pkl', 'rb') as f:
    district_name_encoder = pickle.load(f)
    
# Load the LabelEncoder  for 'medical_region'
with open('medical_fee_region_encoder.pkl', 'rb') as f:
    medical_fee_region_encoder = pickle.load(f)                       
    
# Load the LabelEncoder  for 'gender'
with open('gender_encoder.pkl', 'rb') as f:
       gender_encoder = pickle.load(f)

#carrier name


# Load the LabelEncoder  for 'cause injury category'
#with open('cause_injury_category_encoder.pkl', 'rb') as f:
#       cause_injury_category_enconder = pickle.load(f)

# Load the LabelEncoder  for 'nature injury category'
#with open('nature_injury_category_encoder.pkl', 'rb') as f:
#       nature_injury_category_enconder = pickle.load(f)


# Load the LabelEncoder  for 'body category'
#with open('body_category_encoder.pkl', 'rb') as f:
#       nature_injury_category_enconder = pickle.load(f)



In [3]:
from flask import Flask, render_template, request
import threading
import pandas as pd
import socket
import webbrowser
from datetime import datetime
# Inicializando a aplicação Flask
app = Flask(__name__)

# Inicializando a porta dinamicamente (caso queira usar uma porta diferente)
myPort = 5000  # ou use o código abaixo para porta dinâmica
# with socket.socket() as s:
#     s.bind(('', 0))
#     myPort = s.getsockname()[1]
#print("Running on port:", myPort)
#def encode_features(features, encoders):
#    """
#    Encodes a list of features using the corresponding encoders.
#    
#    :param features: A dictionary of feature names and their values
#    :param encoders: A dictionary of encoders where keys are feature names and values are encoder instances
#    :return: A list of encoded features
#    """
#    encoded_features = []
#    for feature_name, value in features.items():
#        # Apply the corresponding encoder to the feature
#        encoder = encoders.get(feature_name)
#        if encoder:
#            encoded_value = encoder.transform([value])[0]
#            encoded_features.append(encoded_value)
#        else:
#            # Handle case where no encoder is found
#            encoded_features.append(value)  # Just append raw value if no encoding is required
#    return encoded_features

import pickle
import os
from sklearn.preprocessing import LabelEncoder

   



def encoder(mapping, key):
    # Converte a chave para string (para garantir que seja do tipo correto)
    key = str(key)
    
    # Verifica se a chave está no dicionário
    if key in mapping:
        return mapping[key]  # Retorna o valor correspondente
    else:
        return "Not found"


def extract_numbers(code):
    match = re.match(r'^(\d+)', code)
    return int(match.group(1)) if match else None

def check_wage(average_weekly_wage):
    """Determine wage replacement eligibility based on average weekly wage."""
    return 0 if average_weekly_wage > 0 else 1

def check_retirement_status(age):
    """Determine retirement status based on age."""
    retirement_age = 62
    return 0 if age > retirement_age else 1

def check_notification_status(declaration_time):
    """Determine notification status based on declaration time in days."""
    return 1 if declaration_time <= 30 else 0

def check_hearing_date(first_hearing_date):
    if first_hearing_date:
        return 1  # Se o valor for preenchido (não nulo, não vazio)
    else:
        return 0

# Add the label_mapping dictionary
label_mapping = {
    0: '1. CANCELLED',
    1: '2. NON-COMP',
    2: '3. MED ONLY',
    3: '4. TEMPORARY',
    4: '5. PPD SCH LOSS',
    5: '6. PPD NSL',
    6: '7. PTD',
    7: '8. DEATH'
}


# Função para rodar o Flask
def run_app():
    app.run('localhost', debug=True, port=myPort, use_reloader=False)


# Rota para a página inicial (home.html)
@app.route('/')
def home():
    return render_template('home.html')

# Rota para processar os dados do formulário e exibir os resultados (result.html)
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        # Coleta os dados do formulário
        accident_date = request.form['accident-date']
        age_at_injury = float(request.form['age-at-injury'])
        alternative_dispute_resolution = request.form['alternative-dispute-resolution']
        assembly_date = request.form['assembly-date']
        attorney_representative = request.form['attorney-representative']
        average_weekly_wage = float(request.form['average-weekly-wage'])
        birth_year = int(request.form['birth-year'])
        c2_date = request.form['c2-date']
        c3_date = request.form['c3-date']
        carrier_type = request.form['carrier-type'].upper()
        #carrier_name = request.form['carrier-name'].upper()
        county_of_injury = request.form['county-of-injury'].upper()
        covid19_indicator = request.form['agreement-reached']
        district_name = request.form['district-name'].upper()
        first_hearing_date = request.form['first-hearing-date']
        gender = request.form['gender']
        ime4_count = int(request.form['ime4-count'])
        industry_code = request.form['industry-code']
        medical_region = request.form['medical-region']
        wcio_cause_of_injury_code = request.form['wcio-cause-of-injury-code']
        wcio_nature_of_injury_code = request.form['wcio-nature-of-injury-code']
        wcio_part_of_body_code = request.form['wcio-part-of-body-code']
        zip_code = request.form['zip']
        agreement_reached = request.form['agreement-reached']
        wcb_decision = request.form['wcb-decision']
        number_dependents = int(request.form['number_dependents'])

       # date_features = np.array([[accident_date, assembly_date, c2_date, c3_date, first_hearing_date]])
        #accident_date = datetime.strptime(accident_date, '%d-%m-%Y')
        #assembly_date = datetime.strptime(assembly_date, '%d-%m-%Y')
        #c2_date = datetime.strptime(c2_date, '%d-%m-%Y')
        #c3_date = datetime.strptime(c3_date, '%d-%m-%Y')
      
        accident_date = accident_date.strip()
        accident_date = pd.to_datetime(accident_date, format='%Y-%m-%d')
        assembly_date = assembly_date.strip()
        assembly_date = pd.to_datetime(assembly_date, format='%Y-%m-%d')
        c2_date = c2_date.strip()
        c2_date = pd.to_datetime(c2_date, format='%Y-%m-%d')
        c3_date = c3_date.strip()
        c3_date = pd.to_datetime(c3_date, format='%Y-%m-%d')

        first_hearing_date = pd.to_datetime(first_hearing_date, format = '%Y-%m-%d')
        first_hearing_date = check_hearing_date(first_hearing_date)
               
        
        # New Features (using datetime objects)
        declaration_time = (assembly_date - accident_date).days
        receiving_time = (c2_date - assembly_date).days
        receiving_time_c2 = (c2_date - accident_date).days
        wage_replacement_eligibility = check_wage(average_weekly_wage)
        retirement_status = check_retirement_status(age_at_injury)
        notification_status = check_notification_status(declaration_time)

        ###ENCODING BINARY VARIABLES
        # Mapping 'Y'/'yes' to 1, 'N'/'no' to 0
        alternative_dispute_resolution = 1 if alternative_dispute_resolution.lower() in ['y', 'yes'] else 0
        attorney_representative = 1 if attorney_representative.lower() in ['y', 'yes'] else 0
        covid19_indicator = 1 if covid19_indicator.lower() in ['y', 'yes'] else 0
        agreement_reached = 1 if agreement_reached.lower() in ['y', 'yes'] else 0

        carrier_type = encoder(carrier_type_encoder,carrier_type )
        county_of_injury = encoder(county_injury_encoder, county_of_injury)
        district_name= encoder(district_name_encoder, district_name)
        gender = encoder(gender_encoder, gender)
        medical_region = encoder(medical_fee_region_encoder, medical_region)
        
        ###CODES
        industry_code_number = extract_numbers(industry_code)
        cause_of_injury_number = extract_numbers(wcio_cause_of_injury_code)
        nature_of_injury_number = extract_numbers(wcio_nature_of_injury_code)
        part_of_body_number = extract_numbers(wcio_part_of_body_code)


        #cause_injury_category = cause_injury_category_encoder.transform([wcio_cause_of_injury_code])[0]
        #nature_injury_category = nature_injury_category_encoder.transform([wcio_nature_of_injury_code])[0]
        #body_category = body_category_encoder.transform([wcio_part_of_body_code])[0]



        numerical_features = np.array([[age_at_injury, alternative_dispute_resolution, attorney_representative, 
                                        average_weekly_wage, carrier_type, covid19_indicator, county_of_injury,
                                        ime4_count, industry_code_number,  cause_of_injury_number,district_name, gender, medical_region,
                                        nature_of_injury_number, part_of_body_number, number_dependents, declaration_time,
                                        receiving_time, receiving_time_c2, wage_replacement_eligibility, retirement_status,
                                        notification_status]])
        # Scale the numerical features using the loaded scaler
        numerical_features = numerical_features.astype(float)

        scaler.fit(numerical_features)

        scaled_numerical_features = scaler.transform(numerical_features)
        
        # Combine the scaled features with the categorical feature
        #final_features = np.concatenate([scaled_numerical_features, binary_features, cat_features, code_features], axis=1)
        final_features = np.array([[age_at_injury, #carrier_name,
                                   attorney_representative, average_weekly_wage, alternative_dispute_resolution, carrier_type, county_of_injury,district_name, gender, medical_region,
                                 covid19_indicator,  first_hearing_date, ime4_count, 
                                   industry_code_number, cause_of_injury_number, nature_of_injury_number, 
                                   part_of_body_number, number_dependents, 
                                   declaration_time, receiving_time, receiving_time_c2, 
                                   retirement_status, notification_status]])
                                    #, cause_injury_category, nature_injury_category, body_category]])



        # Make the prediction
        prediction = model.predict(final_features)[0]
        
        # Decode the prediction using the label_mapping dictionary
        decision = label_mapping.get(prediction, "Unknown Prediction")
        

        # Cria um dicionário com os valores inseridos
        

        # Passa os dados para o template result.html
        return render_template('result.html', decision=decision)
    
    # Caso o método seja GET, renderiza o formulário (home.html)
    return render_template('home.html')

# Rodando o Flask em um thread
threading.Thread(target=run_app).start()

# Abrindo o app no navegador
webbrowser.open(f"http://127.0.0.1:{myPort}")


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://localhost:5000
Press CTRL+C to quit


True